In [ ]:
import requests
import os
import json
import time
from tqdm import tqdm
from glob import glob
import pandas as pd
import re
import numpy as np
import math

data = pd.read_pickle('input.pkl')
def create_url(id):
    # Specify the usernames that you want to lookup below
    url = "https://api.twitter.com/2/tweets?ids={}&tweet.fields=created_at".format(id)

    return url

def bearer_oauth0(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = 'your key'
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r

def bearer_oauth1(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = 'your key'
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r

ids = data['most_recent_tweet_id'].to_list()
all_data = []
gone = 0
for index, id in tqdm(enumerate(ids)):
    try:

        if True:
            url = create_url((id))
            if gone == 0:
                response = requests.request("GET", url, auth=bearer_oauth0)
                if response.status_code == 429:
                    response = requests.request("GET", url, auth=bearer_oauth1)
                    gone = 1
            elif gone == 1:
                response = requests.request("GET", url, auth=bearer_oauth1)
                if response.status_code == 429:
                    gone = 2
            else:
                print("429 Too Many Requests - Waiting for 15 minutes before retrying...")
                gone = 0
                pd.DataFrame(all_data).to_csv(f'./tweets/{index}.csv', index=False)
                all_data = []
                time.sleep(900)  # 15 minute

            if response.status_code == 200:
                json_response = response.json()
                json_response['tweet_id_query'] = id
                all_data.append(json_response)
                

            elif response.status_code != 200:
                print(
                    "Request returned an error: {} {}".format(
                        response.status_code, response.text
                    )
                )
    except Exception as e:
        print(e)
